<a href="https://colab.research.google.com/github/ako1983/awesome-compound-ai-systems/blob/main/Friendli_GIFpt_Challenge_1112.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Friendli GIFpt Challenge**

# **Friendli GIF chatbot**

## ***Getting Started***


1.   Head to [*https://suite.friendli.ai*](https://suite.friendli.ai/signup/self-register?product=serverless-endpoints), and create an account.
2.   Grab a [**`FRIENDLI_TOKEN`**](https://suite.friendli.ai/user-settings/tokens) to use Friendli Serverless Endpoints for the assistant.
3.   Install dependencies

API KEY

In [1]:
pip install openai gradio requests --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.7 MB/s eta 0:00:00


### 🚀 *Build and visualize your assistant!*
1. Do the function call using meta-llama-3.1-70b-instruct
2. Pass the output image as an input of llama 3.2 vision model

In [ ]:
import requests
import gradio as gr
import json
from getpass import getpass
from openai import OpenAI

css = """
.gradio-container {
    max-width: 800px !important;
    margin-top: 100px !important;
}
.pending {
    display: none !important;
}
.sm {
    box-shadow: None !important;
}
"""

GIFPY_API_KEY = "AZlZzMlDoD27PLV9QPJ5GvU7UvpuLgHt"
FRIENDLI_TOKEN = getpass("FRIENDLI_TOKEN:")

friendli_client = OpenAI(
    base_url="https://inference.friendli.ai/v1",
    api_key=FRIENDLI_TOKEN
)

friendli_beta_client = OpenAI(
    base_url="https://inference.friendli.ai/beta",
    api_key=FRIENDLI_TOKEN
)

def search_gif(keyword):
    response = requests.get(f"https://api.giphy.com/v1/gifs/search?api_key={GIFPY_API_KEY}&q={keyword}&limit=1&offset=0&rating=g&lang=en")

    if response.status_code == 200:
        data = response.json()
        if data['data']:
            return data['data'][0]['images']['original']['url']

    return None



In [ ]:
def chatbot_response(message, history):
    # message -> extract keyword for a GIF using llm
    tools = [{
        "type": "function",
        "function": {
            "name": "search_gif",
            "description": "Search for a GIF using the given keyword.",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "A keyword for a GIF that fits in the conversation.",
                    },
                },
            },
        },
    }]

    messages = [{"role": "system", "content": "Find a keyword for a GIF that fits in the conversation."}]
    for user, chatbot in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": chatbot})
    messages.append({"role": "user", "content": message})

    tool_response = friendli_client.chat.completions.create(
        model="meta-llama-3.1-70b-instruct",  # Fill in the model name here
        messages=messages,
        tools=tools,
    )
    keyword = message
    if tool_response.choices[0].message.tool_calls:
        tool_call = tool_response.choices[0].message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)
        keyword = arguments.get('keyword')

    gif_url = search_gif(keyword)
    messages_for_response = []
    if gif_url:
        # messages_for_response.append({
        #     "role": "system",
        #     "content": "The input image is actually what you will answer with, answer properly."
        # })
        for user, chatbot in history:
            messages_for_response.append({"role": "user", "content": user})
            messages_for_response.append({"role": "assistant", "content": chatbot})
        messages_for_response.append({
            "role": "user", "content": [
                {"type": "text", "text": message + "\n\nThe input image is actually what you will answer with, answer properly."},
                {"type": "image_url", "image_url": {"url": gif_url}}
            ]
        })
    else:
        for user, chatbot in history:
            messages_for_response.append({"role": "user", "content": user})
            messages_for_response.append({"role": "assistant", "content": chatbot})
        messages_for_response.append({"role": "user", "content": message})

    reply_response = friendli_beta_client.chat.completions.create(
        model="llama-3.2-11b-vision-instruct",  # Fill in the model name here too
        messages=messages_for_response,  # Fill in the variable for messages here
        timeout=60,
        stream=True,
    )
    reply = ""
    for chunk in reply_response:
        if chunk.choices is not None:
            reply += chunk.choices[0].delta.content or ""
    return reply, gif_url



def chat_interface_fn(message, history):
    reply, gif = chatbot_response(message, history)
    if gif:
        return [(message, f"{reply}\n![GIF]({gif})")]
    return [(message, f"{reply}")]


with gr.Blocks(theme=gr.themes.Soft(), css=css) as agent:
    def respond(message, history: list):
        response = chat_interface_fn(message, history)
        history.extend(response)
        return history, history

    chat = gr.Chatbot()
    textbox = gr.Textbox(show_label=False, placeholder="Type a message...")
    textbox.submit(respond, [textbox, chat], [chat, chat])

agent.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a5458be3eb1d27a4ed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
